In [1]:
import os

print("Files and folders in the current directory:")
print(os.listdir('.'))


Files and folders in the current directory:
['(Assignment 1 & 2)Regular Expression.docx.pdf', '2021cs14_quiz1.pdf', 'ASS12.docx', 'Assignment 4_Edit distance & Sequence Alignment.docx.pdf', 'Assignment1_2.ipynb', 'harrypotter.txt', 'harry_potter', 'HPGEN.ipynb', 'NLP_Slides', 'training_checkpoints', 'Tuning Transformer for Summary Generation T5.ipynb', '_(Assignment 3)Sentence Segmentation.docx.pdf']


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

for dirname, _, filenames in os.walk('./harry_potter/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./harry_potter/1SorcerersStone.txt
./harry_potter/2ChamberofSecrets.txt
./harry_potter/3ThePrisonerOfAzkaban.txt
./harry_potter/4TheGobletOfFire.txt
./harry_potter/5OrderofthePhoenix.txt
./harry_potter/6TheHalfBloodPrince.txt
./harry_potter/7DeathlyHollows.txt


In [3]:
import tensorflow as tf
import numpy as np
import os
import time

In [ ]:
# Merge all Harry Potter books into a single file
with open('harrypotter.txt', 'w', encoding='utf-8') as outfile:
    for file in files:
        with open(file, encoding='utf-8') as infile:
            outfile.write(infile.read())

# Read the merged text file
with open('harrypotter.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(f'Length of text: {len(text)} characters')


UnicodeDecodeError: 'charmap' codec can't decode byte 0x90 in position 970493: character maps to <undefined>

In [6]:
# Taking a look at the text
print(text[:300])

Harry Potter and the Sorcerer's Stone 

CHAPTER ONE 

THE BOY WHO LIVED 

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you'd expect to be involved in anything strange or mysterious, because they 


In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

106 unique characters


In [8]:
# Creating a mapping from unique characters to indices
char2index = {u:i for i, u in enumerate(vocab)}
index2char = np.array(vocab)

text_as_int = np.array([char2index[c] for c in text])

print(text_as_int)

[39 64 81 ... 75 75 15]


In [9]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} -- characters mapped to int -- > {}'.format(repr(text[:13]), text_as_int[:13]))

'Harry Potter ' -- characters mapped to int -- > [39 64 81 81 88  3 47 78 83 83 68 81  3]


In [10]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
#sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

for i in char_dataset.take(5):
  print(index2char[i.numpy()])


H
a
r
r
y


In [11]:
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(index2char[item.numpy()])))



"Harry Potter and the Sorcerer's Stone \n\nCHAPTER ONE \n\nTHE BOY WHO LIVED \n\nMr. and Mrs. Dursley, of nu"
'mber four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They'
" were the last people you'd expect to be involved in anything strange or mysterious, because they jus"
"t didn't hold with such nonsense. \n\nMr. Dursley was the director of a firm called Grunnings, which ma"
'de drills. He was a big, beefy man with hardly any neck, although he did have a very large mustache. '


In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [14]:
for input_example, target_example in dataset.take(1):
    print("Input shape:", input_example.shape)
    print("Target shape:", target_example.shape)

Input shape: (64, 100)
Target shape: (64, 100)


In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 300 #256

# Number of RNN units
rnn_units1 = 1024
rnn_units2 = 1024
rnn_units=[rnn_units1, rnn_units2]
print(vocab_size)

106


In [16]:
seq_length = 100  # Define sequence length explicitly

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size, seq_length):
    model = tf.keras.Sequential([
        # Define the input layer with batch input shape
        tf.keras.layers.InputLayer(batch_input_shape=(batch_size, seq_length)),

        # Embedding layer
        tf.keras.layers.Embedding(vocab_size, embedding_dim),

        # First GRU layer
        tf.keras.layers.GRU(rnn_units[0],
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        # Second GRU layer
        tf.keras.layers.GRU(rnn_units[1],
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        # Output layer
        tf.keras.layers.Dense(vocab_size)
    ])

    return model


In [18]:
seq_length = 100  # Define your sequence length

model = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE,
    seq_length=seq_length
)


In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (64, 100, 300)              │          31,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (64, 100, 1024)             │       4,073,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (64, 100, 1024)             │       6,297,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (64, 100, 106)              │         108,650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,511,522 (40.10 MB)

 Trainable params: 10,511,522 (40.10 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [21]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [22]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [23]:
EPOCHS=1

In [24]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

In [25]:
latest_check= tf.train.latest_checkpoint(checkpoint_dir)

In [28]:
import os
print(os.listdir(checkpoint_dir))

# Manually assign the checkpoint file path
latest_check = os.path.join(checkpoint_dir, 'ckpt_1.weights.h5')

# Ensure the file exists
if not os.path.exists(latest_check):
    raise FileNotFoundError(f"Checkpoint file not found: {latest_check}")


['ckpt_1.weights.h5']


In [29]:
# Load the weights into the model
seq_length = 100
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1, seq_length=seq_length)

# Check if the latest_check variable is not None before loading weights
if latest_check is not None:
    model.load_weights(latest_check)
else:
    print("No checkpoint file found. Starting with a fresh model.")


# Rebuild the model for inference
#model.build(tf.TensorShape([1, None]))

In [30]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (1, 100, 300)               │          31,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_4 (GRU)                          │ (1, 100, 1024)              │       4,073,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_5 (GRU)                          │ (1, 100, 1024)              │       6,297,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (1, 100, 106)               │         108,650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,511,522 (40.10 MB)

 Trainable params: 10,511,522 (40.10 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# Ensure dataset is correctly batched
BATCH_SIZE = 64
BUFFER_SIZE = 10000

# Shuffle and batch the dataset
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Calculate split index for training and testing
split_index = int(0.8 * len(list(dataset)))
test_dataset = dataset.skip(split_index)

# Ensure the model is compiled
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Check shape of dataset before passing it into the model
for example in test_dataset.take(1):
    input_example, target_example = example
    print(input_example.shape)  # Verify shape
    print(target_example.shape)  # Verify shape


(64, 64, 100)
(64, 64, 100)


In [32]:
test_dataset = test_dataset.batch(1)
test_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(None, 64, 64, 100), dtype=tf.int64, name=None))>

In [ ]:

# Evaluate the model on the test dataset
evaluation_metrics = model.evaluate(test_dataset, verbose=1)


import matplotlib.pyplot as plt

# Print the evaluation results
print("Evaluation Results:")
print(f"Loss: {evaluation_metrics[0]}")
print(f"Accuracy: {evaluation_metrics[1]}")

# Plot the evaluation metrics
metrics = ['Loss', 'Accuracy']
values = evaluation_metrics

plt.figure(figsize=(10, 5))
plt.bar(metrics, values, color=['blue', 'green'])
plt.xlabel('Metrics')
plt.ylabel('Values')
plt.title('Model Evaluation Metrics')
plt.show()

In [33]:
def generate_text(model, start_string):
    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2index[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low results in more predictable text.
    # Higher results in more surprising text.
    # Experiment to find the best setting.
    scaling = 0.5  # 1

    # Resetting the states of RNN layers
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.GRU):
            layer.reset_states()

    for i in range(num_generate):
        predictions = model(input_eval)
        # Remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # Using a categorical distribution to predict the character returned by the model
        predictions = predictions / scaling
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(index2char[predicted_id])

    return start_string + ''.join(text_generated)


In [34]:
print(generate_text(model, start_string=u"Dumbledore "))

Dumbledore was still painting the first time and falling in the castle and the only one who had been a birt of under Harry's head.
"I don't know what to do it."
"It was the one who were pointing to the grass and still still standing in the castle all the picture of the common room, and Hermione had been blacked in his hands, the stone still standing in sides, but Harry could not help the potion in the end of the sound of the fear of the castle, the stone steps and sund them to him to take a small black door.
"But I think the wizard said Ron gave a pair of sight behind him. "I beg in the castle that many mirror. Harry said to see the silence that spend like many into the corner that he was important to tee the staircase with the tent to him to Harry.
"But what do you seen him - years ago," said Harry, his last sing in the same time the point in the end of the cold small blink and that where he had been concentrated in a few moments were still more than she had the snake of the castle wi

In [35]:
print(generate_text(model, start_string=u"Harry killed Ron  "))

Harry killed Ron  standing as they left the tiny sides, and say Lily, the last little shock and still still still still still so far off it in front of the steps to the corner and still still with a small shoulders for a moment the common room. He had not seemed to him to see the marble still stormed in his back to the side of the same moment, the only sound of the castle was the only one who cleared behind him.
"Perhaps it did not in the corridor but he said, and he felt like the ground of the Prime Minister and the grass as he pulled out the family to make a man was a long time whispered in the door to the pain of the wall back and the many parchment, and she said impressive in his great seconds had a conversation had been closed his eyes in her hand. He could not remember in the door.
"Well, the first time you think you have to look at him to get the connection of the stone smoke and turned in the corner and hin in the face stand at the room and the door stood shaking his eyes with 

In [36]:
print(generate_text(model, start_string=u"Harry killed Ron  "))

Harry killed Ron  came to the corridor and his shoulder and feel that he had the goblin person her sleep and shook his face as he said, "The Death Eater was still wearing its sides of the castle, and he was still still going to him that might have been before the marble and shook his face from the sofath time behind them in the head in a long time that was the shider of his face with him. He was still eyes from the gate and little high and this time they seemed to have a way back in the countryside of the pool and streaming out of the front of the Hallows -- "
"I didn’t have to do it all this and tears to the curtains and inside the air again. He left him to discuss them a bit of back to him, like uncomfortable to the common room with a many place where he said, and the visit that it was not so on the exception that he was him and say in the castle with him that the man common room was silently so that his life could have to mention the steps to Harry Potterwatch with the battle black 

In [37]:
print(generate_text(model, start_string=u"Hermoine Killed Snape  "))

Hermoine Killed Snape  behind them as he said, and the goblin side him back, planning to him he had been relieved it open. The parchment of the sun. Harry said in a little silence. "He did not leave the story side of the contrary, and Harry saw the bloodship was still standing at the sound of her of the windows and the same boy was leading Ron and Hermione came to his feet as they said, and where they were saying the stone still seemed to say Tonks had been like the sword to the first bed, and the brightened and took and said, "This is it was a long time we've got a place where the common room with him to make a connection by the same time the man who was remembering a mistake. Harry said as they were still at the end of the garden pause. "I don’t mind the curse that he was still to him, on the same minutes to him, but I think I leave the parchment to the same moment, the wand hold that had he had him to see them in the darkness and the swirling village remained into the desperate the 